# Setup

In [ ]:
exp_directory = 'results'
agent_name = 'DoubleDeepQAgent'
game_name = 'Breakout'
render_mode='rgb_array'
fps = None # 60

### Global Modules

In [ ]:
%matplotlib inline
import pandas as pd
import os
import sys
from gym.wrappers import Monitor

### Local Modules

In [ ]:
import base
from src.agents import (
    DeepQAgent,
    DoubleDeepQAgent
)
from src.environment.atari import build_atari_environment

## Constants

In [ ]:
# a mapping of string names to agents
agents = {
    'DeepQAgent': DeepQAgent,
    'DoubleDeepQAgent': DoubleDeepQAgent,
}

In [ ]:
exp_directory = '{}/{}/{}'.format(exp_directory, game_name, DoubleDeepQAgent.__name__)
if not os.path.exists(exp_directory):
    raise ValueError('{} not found!'.format(repr(exp_directory)))
# set up the weights file
weights_file = '{}/weights.h5'.format(exp_directory)
# make sure the weights exist
if not os.path.exists(weights_file):
    raise ValueError('{} not found!'.format(repr(weights_file)))

# Environment

In [ ]:
env = build_atari_environment(game_name)
env = Monitor(env, '{}/monitor'.format(exp_directory), force=True)

In [ ]:
env.observation_space

In [ ]:
env.action_space

# Agent

In [ ]:
agent = agents[agent_name](env, 
    render_mode=render_mode, 
    replay_memory_size=0
)
agent.model.load_weights(weights_file)
agent

In [ ]:
scores = agent.play(fps=fps)
scores = pd.Series(scores)
scores

In [ ]:
_ = scores.hist()

In [ ]:
scores.describe()